# TF-IDF Search Using Cosine Similarity

In [6]:
import pandas as pd
import spacy
from collections import Counter

In [4]:
text_df = pd.read_json("processed_text.json")
text_df.head()

,title,text,url,processed_text
0,Pandemic,"A pandemic (from Greek πᾶν, pan, ""all"" and δῆμ...",https://en.wikipedia.org/wiki/Pandemic,"[pandemic, greek, pan, demo, people, epidemic,..."
1,Epidemiology of HIV/AIDS,"HIV/AIDS, or Human Immunodeficiency Virus, is ...",https://en.wikipedia.org/wiki/Epidemiology_of_...,"[hiv, aid, human, immunodeficiency, virus, con..."
2,Antonine Plague,"The Antonine Plague of 165 to 180 AD, also kno...",https://en.wikipedia.org/wiki/Antonine_Plague,"[plague, 165, 180, ad, know, plague, galen, ga..."
3,Basic reproduction number,"In epidemiology, the basic reproduction number...",https://en.wikipedia.org/wiki/Basic_reproducti...,"[epidemiology, basic, reproduction, number, ba..."
4,Bills of mortality,Bills of mortality were the weekly mortality s...,https://en.wikipedia.org/wiki/Bills_of_mortality,"[bill, mortality, weekly, mortality, statistic..."


In [46]:
all_text_list = list(text_df["processed_text"])
corpus = [word for sublist in all_text_list for word in sublist]
len(Counter(corpus))


1408

In [32]:
def computeTF(wordDict):
    tfDict = {}
    WordsCount = len(wordDict)
    for word, count in wordDict.items():
        tfDict[word] = count / float(WordsCount)
    return tfDict

## Term Frequency

**The number of times a word appears in a document divided by the total number of words in the document. Every document has its own term frequency.**

In [43]:
def computeTF(document):
    return(Counter(document))

In [45]:
text_df["term_frequency"] = text_df["processed_text"].apply(computeTF)

## Inverse Document Frequency

The log of the number of documents divided by the number of documents that contain the word w. Inverse data frequency determines the weight of rare words across all documents in the corpus.

In [53]:
# Compute Inverse Document Frequency
def build_tf_idf(string, vocab):
    idfDict = dict.fromkeys(vocab, 0)
    for document in text_df.processed_text:
        for word in list(set(document)):
            idfDict[word] += 1


In [ ]:
def computeIDF(document, corpus):
    import math
    N = len(corpus)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict